<a href="https://colab.research.google.com/github/sowmyamanojna/CS6024-Algorithmic-Approaches-to-Computational-Biology-Project/blob/master/vae_tybalt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import io

import keras
import tensorflow as tf
from keras.layers import Input, Dense, Lambda, Layer, Activation
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import backend as K
from keras import metrics, optimizers
from keras.callbacks import Callback
from tensorflow.keras import losses

import pydot
import graphviz
from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [ ]:
%matplotlib inline
plt.style.use('seaborn-notebook')
sns.set(style="white", color_codes=True)
sns.set_context("paper", rc={"font.size":14,"axes.titlesize":15,"axes.labelsize":20,'xtick.labelsize':14, 'ytick.labelsize':14})

In [ ]:
# Function for reparameterization trick to make model differentiable
def sampling(args):
    
    import tensorflow as tf
    # Function with args required for Keras Lambda function
    z_mean, z_log_var = args

    # Draw epsilon of the same shape from a standard normal distribution
    epsilon = K.random_normal(shape=tf.shape(z_mean), mean=0.,
                              stddev=epsilon_std)
    
    # The latent vector is non-deterministic and differentiable
    # in respect to z_mean and z_log_var
    z = z_mean + K.exp(z_log_var / 2) * epsilon
    return z

class CustomVariationalLayer(Layer):
    """
    Define a custom layer that learns and performs the training
    This function is borrowed from:
    https://github.com/fchollet/keras/blob/master/examples/variational_autoencoder.py
    """
    def __init__(self, **kwargs):
        # https://keras.io/layers/writing-your-own-keras-layers/
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)

    def vae_loss(self, x_input, x_decoded):
        reconstruction_loss = original_dim * metrics.binary_crossentropy(x_input, x_decoded)
        kl_loss = - 0.5 * K.sum(1 + z_log_var_encoded - K.square(z_mean_encoded) - 
                                K.exp(z_log_var_encoded), axis=-1)
        return K.mean(reconstruction_loss + (K.get_value(beta) * kl_loss))

    def call(self, inputs):
        x = inputs[0]
        x_decoded = inputs[1]
        loss = self.vae_loss(x, x_decoded)
        self.add_loss(loss, inputs=inputs)
        # We won't actually use the output.
        return x

class WarmUpCallback(Callback):
    def __init__(self, beta, kappa):
        self.beta = beta
        self.kappa = kappa
    # Behavior on each epoch
    def on_epoch_end(self, epoch, logs={}):
        if K.get_value(self.beta) <= 1:
            K.set_value(self.beta, K.get_value(self.beta) + self.kappa)

def get_model_summary(model):
    stream = io.StringIO()
    model.summary(print_fn=lambda x: stream.write(x + '\n'))
    summary_string = stream.getvalue()
    stream.close()
    return summary_string

In [ ]:
np.random.seed(123)

In [ ]:
pcos_df = pd.read_csv('common_normalized.csv')
pcos_df = pcos_df.drop(['sample_id'], axis=1)
# Split 10% test set randomly
test_set_percent = 0.1
pcos_test_df = pcos_df.sample(frac=test_set_percent)
pcos_train_df = pcos_df.drop(pcos_test_df.index)
print(pcos_train_df.head(2))
print(pcos_test_df.head(2))

In [ ]:
# Set hyper parameters
original_dim = pcos_df.shape[1]
latent_dim = 100

batch_size = 4
# epochs = 50
epochs = 10
learning_rate = 0.0005

epsilon_std = 1.0
beta = K.variable(0)
kappa = 1

In [ ]:
# Input place holder for PCOS data with specific input size
pcos_input = Input(shape=(original_dim, ))

# Input layer is compressed into a mean and log variance vector of size `latent_dim`
# Each layer is initialized with glorot uniform weights and each step (dense connections,
# batch norm, and relu activation) are funneled separately
# Each vector of length `latent_dim` are connected to the input tensor
z_mean_dense_linear = Dense(latent_dim, kernel_initializer='glorot_uniform')(pcos_input)
z_mean_dense_batchnorm = BatchNormalization()(z_mean_dense_linear)
z_mean_encoded = Activation('relu')(z_mean_dense_batchnorm)

z_log_var_dense_linear = Dense(latent_dim, kernel_initializer='glorot_uniform')(pcos_input)
z_log_var_dense_batchnorm = BatchNormalization()(z_log_var_dense_linear)
z_log_var_encoded = Activation('relu')(z_log_var_dense_batchnorm)

# return the encoded and randomly sampled z vector
# Takes two keras layers as input to the custom sampling function layer with a `latent_dim` output
z = Lambda(sampling, output_shape=(latent_dim, ))([z_mean_encoded, z_log_var_encoded])

In [ ]:
# The decoding layer is much simpler with a single layer and sigmoid activation
decoder_to_reconstruct = Dense(original_dim, kernel_initializer='glorot_uniform', activation='sigmoid')
pcos_reconstruct = decoder_to_reconstruct(z)

In [ ]:
adam = optimizers.Adam(lr=learning_rate)
vae_layer = CustomVariationalLayer()([pcos_input, pcos_reconstruct])
vae = Model(pcos_input, vae_layer)
# vae.compile(optimizer=adam, loss=None, loss_weights=[beta])
vae.compile(optimizer=adam, loss=losses.BinaryCrossentropy, loss_weights=[beta])

vae_model_summary_string = get_model_summary(vae)

with open('vae_model.txt', 'a') as f:
  f.write(vae_model_summary_string)
  f.write('\n')

In [ ]:
tf.config.experimental_run_functions_eagerly(True)
hist = vae.fit(np.asarray(pcos_train_df).astype('float32'),
               shuffle=True,
               epochs=epochs,
               batch_size=batch_size,
               validation_data=(np.asarray(pcos_test_df).astype('float32'), None),
               callbacks=[WarmUpCallback(beta, kappa)])

In [ ]:
# Visualize training performance
history_df = pd.DataFrame(hist.history)
ax = history_df.plot()
ax.set_xlabel('Epochs')
ax.set_ylabel('VAE Loss')
fig = ax.get_figure()
fig.savefig("hist_plot_file.png")

In [ ]:
# Model to compress input
encoder = Model(pcos_input, z_mean_encoded)

In [ ]:
# Encode into the hidden/latent representation - and save output
encoded_pcos_df = encoder.predict_on_batch(pcos_df)
encoded_pcos_df = pd.DataFrame(encoded_pcos_df, index=pcos_df.index)

encoded_pcos_df.columns.name = 'sample_id'
encoded_pcos_df.columns = encoded_pcos_df.columns + 1
encoded_file = 'encoded_pcos_onehidden_warmup_batchnorm.tsv'
encoded_pcos_df.to_csv(encoded_file, sep='\t')

In [ ]:
# build a generator that can sample from the learned distribution
decoder_input = Input(shape=(latent_dim, ))  # can generate from any sampled z vector
_x_decoded_mean = decoder_to_reconstruct(decoder_input)
decoder = Model(decoder_input, _x_decoded_mean)

In [ ]:
encoder_model_file = 'encoder_onehidden_vae.hdf5'
decoder_model_file = 'decoder_onehidden_vae.hdf5'

encoder.save(encoder_model_file)
decoder.save(decoder_model_file)

In [ ]:
# What are the most and least activated nodes
sum_node_activity = encoded_pcos_df.sum(axis=0).sort_values(ascending=False)

with open('10_most_and_least_activity_nodes.txt', 'a') as f:
  f.write(sum_node_activity.head(10).to_string())
  f.write('\n')
  f.write(sum_node_activity.tail(10).to_string())

In [ ]:
# Histogram of node activity for all 100 latent features
sum_node_activity.hist()
plt.xlabel('Activation Sum')
plt.ylabel('Count')
plt.savefig('node_activity_hist.png')

In [ ]:
# Example of node activation distribution for the first two latent features
plt.figure(figsize=(6, 6))
plt.scatter(encoded_pcos_df.iloc[:, 1], encoded_pcos_df.iloc[:, 2])
plt.xlabel('Latent Feature 1')
plt.ylabel('Latent Feature 2')
plt.savefig('node_activation_2_latent.png')

In [ ]:
# How well does the model reconstruct the input data
input_pcos_reconstruct = decoder.predict(np.array(encoded_pcos_df))
input_pcos_reconstruct = pd.DataFrame(input_pcos_reconstruct, index=pcos_df.index,
                                        columns=pcos_df.columns)
input_pcos_reconstruct.head(2)

In [ ]:
reconstruction_fidelity = pcos_df - input_pcos_reconstruct

gene_mean = reconstruction_fidelity.mean(axis=0)
gene_abssum = reconstruction_fidelity.abs().sum(axis=0).divide(pcos_df.shape[0])
gene_summary = pd.DataFrame([gene_mean, gene_abssum], index=['gene mean', 'gene abs(sum)']).T
gene_summary.sort_values(by='gene abs(sum)', ascending=False).head()

In [ ]:
# Mean of gene reconstruction vs. absolute reconstructed difference per sample
g = sns.jointplot('gene mean', 'gene abs(sum)', data=gene_summary)
g.savefig('mean_gene_reconstructed_abs.png')